In [71]:
import seaborn as sns

In [72]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [73]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [75]:
from sklearn.preprocessing import LabelEncoder

In [76]:
encoder = LabelEncoder()

In [77]:
df['time']=encoder.fit_transform(df['time'])

In [78]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [79]:
df['time'].value_counts() # time is dependent feature

0    176
1     68
Name: time, dtype: int64

In [80]:
df.shape

(244, 7)

In [81]:
X = df.drop(labels = ['time'],axis=1)

In [82]:
y = df['time']

In [83]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [84]:
from sklearn.impute import SimpleImputer# handling missing values
from sklearn.preprocessing import OneHotEncoder# handling categorical features
from sklearn.preprocessing import StandardScaler# feature scaling
from sklearn.pipeline import Pipeline

In [85]:
numerical_cols = ['total_bill','tip','size']
cateogrical_cols = ['sex','smoker','day']

In [86]:
## Feature Engineering Automation
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), ##missing values
        ('scaler',StandardScaler())## feature scaling 
    ]

)

#categorical Pipeline
cat_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')), ## handling Missing values
                ('onehotencoder',OneHotEncoder()) ## Categorical features to numerical
                ]

            )  

In [87]:
from sklearn.compose import ColumnTransformer

In [68]:
preprocessor = ColumnTransformer([('num_pipeline',num_pipeline,numerical_cols),
                                  ('cat_pipeline',cat_pipeline,cateogrical_cols)
                                 ]
                                )

In [90]:
X_train=preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
 

In [113]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [114]:
models = {
 'Random_Forest': RandomForestClassifier(),
 'Decision_Tree': DecisionTreeClassifier(),
 'Support_vector': SVC()
}

In [111]:
from sklearn.metrics import accuracy_score

In [105]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # train model
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test,y_pred)
        report[list(models.keys())[i]] = accuracy
    return report
        

In [115]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random_Forest': 0.9591836734693877,
 'Decision_Tree': 0.9387755102040817,
 'Support_vector': 0.9591836734693877}

In [116]:
classifier = RandomForestClassifier()

In [117]:
## Hypeparameter Tuning
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [118]:
from sklearn.model_selection import GridSearchCV

In [119]:
clf = GridSearchCV(classifier,param_grid=params,cv=5,scoring = 'accuracy',verbose=3)

In [120]:
clf.fit(X_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.949 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.923 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.923 total time=   0.2s
[CV 1/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.974 total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.949 total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.974 total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.923 total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.923 total time=   0.3s
[CV 1/5] END crite

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 10, None],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy', verbose=3)

In [121]:
clf.best_params_

{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 200}

In [122]:
y_pred_hyper = clf.predict(X_test)

In [124]:
print(accuracy_score(y_test,y_pred_hyper))

0.9591836734693877
